In [ ]:
import sys
sys.path.append('/Users/dan/Code/LLM/llm_analyst')

In [ ]:
from llm_analyst.core.research_state import ResearchState
research_state = ResearchState()

In [ ]:
from llm_analyst.core.config import Config
try:
    config = Config()
    print(config)
except Exception as e:
    print(f"Error {e}")

In [ ]:
from llm_analyst.document.document import DocumentLoader

local_doc_dir="/Users/dan/Downloads/amy_papers"
document_loader = DocumentLoader(local_doc_dir)
docs = await document_loader.load()
docs

In [ ]:
from llm_analyst.core.config import Config
from llm_analyst.core.research_editor import LLMEditor
from llm_analyst.core.research_analyst import LLMAnalyst
from llm_analyst.core.research_writer import LLMWriter
from llm_analyst.core.research_publisher import LLMPublisher
from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone
query = "What happened in the latest burning man floods?"
config_params = {}
config_params = {
    "internet_search" :{"default_val":"ddg_search"},
    "llm_provider"    :{"default_val":"openai"},
    "llm_model"       :{"default_val":"gpt-4o-2024-05-13"},
}

#    "llm_model"       :{"default_val":"mixtral-8x7b-32768"},
#    "llm_model"       :{"default_val":"gemma-7b-it"},
#    "llm_model"       :{"default_val":"llama3-8b-8192"},
#    "llm_model"       :{"default_val":"llama3-70b-8192"},
#    "llm_model"       :{"default_val":"gpt-4o-2024-05-13"},


config = Config()
config._set_values_for_config(config_params)


In [ ]:
llm_editor = LLMEditor(query, config = config)
detailed_report_state = await llm_editor.create_detailed_report()
print(detailed_report_state.final_report_md)

In [ ]:
await LLMPublisher.init(detailed_report_state).publish_to_pdf_file()

In [ ]:

llm_analyst = LLMAnalyst(query,config = config)
primary_research = choose_agent_dict = await llm_analyst.conduct_research()
print("="*40)
print(primary_research)
subtopics = await llm_analyst.select_subtopics()
print("="*40)
print(subtopics)
report_state = await llm_analyst.write_report()
print("="*40)
print(report_state.report_md)


In [ ]:
from llm_analyst.core.research_writer import LLMWriter

subtopic_reports = []
subtopics_report_body = ""

for subtopic in subtopics:
    print(f"Researching {subtopic}")
    subtopic_assistant = LLMAnalyst(
        active_research_topic = subtopic,
        report_type = "subtopic_report",
        main_research_topic = primary_research.active_research_topic,
        visited_urls = primary_research.visited_urls,
        agents_role_prompt =  primary_research.agents_role_prompt,
        agent_type = primary_research.agent_type
    )
    subtopic_assistant.research_findings = primary_research.research_findings
    subtopic_assistant.report_headings = primary_research.report_headings
     
    subtopic_research = await subtopic_assistant.conduct_research()
    print(f"Writing {subtopic}")
    subtopic_report = await subtopic_assistant.write_report()

    primary_research.research_findings = subtopic_assistant.research_findings
    primary_research.visited_urls.update(subtopic_assistant.visited_urls)
    primary_research.report_headings.append(
        {
            "subtopic task": subtopic_assistant.active_research_topic,
            "headers": LLMWriter.init(subtopic_report).extract_headers()
        }
    )
    subtopic_reports.append({ "topic": subtopic_assistant.active_research_topic, "report": subtopic_report.report_md })
    primary_research.report_md += "\n\n\n" + subtopic_report.report_md

#llm_writer = LLMWriter.init(primary_research)
#introduction = await llm_writer.write_introduction()
    

In [ ]:
introduction = await LLMWriter.init(primary_research).write_introduction()
print(introduction)

In [ ]:
toc = await LLMWriter.init(primary_research).write_table_of_contents()
print(toc)

In [ ]:
references = LLMWriter.init(primary_research).write_references()
print(references)

In [ ]:
from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone

context = "This context"
question = "What is the meaning of life?"
total_words = 2000
report_format = "APA"
datetime_now = datetime.now().strftime('%B %d, %Y')

prompts = Prompts()
prompt = prompts.get_prompt("generate_report_prompt",
                            context=context,
                            question=question,
                            total_words=total_words,
                            report_format=report_format,
                            datetime_now=datetime_now)
print(prompt)


In [ ]:
from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone

context = "This context"
question = "What is the meaning of life?"
total_words = 2000
task = "What is the meaning of life?"
max_iterations = 5
datetime_now = datetime.now().strftime('%B %d, %Y')

prompts = Prompts()
prompt = prompts.get_prompt("generate_resource_report_prompt",
                            context=context,
                            question=question,
                            total_words=total_words)
print(prompt)




In [ ]:

from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone

prompts = Prompts()
prompt = prompts.get_prompt("auto_agent_instructions")
print(prompt)



In [ ]:
import inspect

def my_trace_log(func):
    def wrapper(*args, **kwargs):
        function_name = inspect.currentframe().f_code.co_name
        print(f"TRACE: Entering {func.__name__}")
        result = func(*args, **kwargs)
        print(f"TRACE: Exiting {func.__name__}")
        return result
    return wrapper

@my_trace_log
def hello_world():
    print("Hello World")

# Example usage
hello_world()


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from llm_analyst.core.exceptions import LLMAnalystsException
deterministic_temp=0
role = "You are a well-informed AI news analyst assistant. Your primary goal is to provide comprehensive, accurate, unbiased, and well-structured news reports based on the latest events and developments."
report_prompt = "Information: context data....\n\nUsing the above information, answer the following query or task: 'What happened in the latest burning man floods?' in a detailed report -- The report should focus on the answer to the query, should be well structured, informative, in-depth and comprehensive, with facts and numbers if available and a minimum of 1000 words.\nYou should strive to write the report as long as you can using all relevant and necessary information provided.\nYou must write the report with markdown syntax.\nUse an unbiased and journalistic tone.\nYou MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.\nYou MUST write all used source urls at the end of the report as references, and make sure not to add duplicated sources, but only one reference for each.\nEvery url should be hyperlinked: [url website](url)\nAdditionally, you MUST include hyperlinks to the relevant URLs wherever they are referenced in the report : \n\n       e.g.:\n            # Report Header\n\n            This is a sample text. ([url website](url))\n\nYou MUST write the report in APA format.\nCite search results using inline notations.\nOnly cite the most relevant results that answer the query accurately.\nPlace these citations at the end of the sentence or paragraph that reference them.\nPlease do your best, this is very important to my career.\nAssume that the current date is May 21, 2024"
messages=[
        {"role": "system", "content": "role"},
        {"role": "user",   "content": "report_prompt"}]
            
#api_key = os.environ["OPENAI_API_KEY"]
api_key = os.environ["GROQ_API_KEY"]
temperature = 0
#model = "gpt-4o-2024-05-13"
model = "llama3-70b-8192"
max_tokens=4000
llm = ChatGroq(
            model = model,
            temperature = temperature,
            max_tokens = max_tokens,
            api_key = api_key
        )


In [ ]:
config_params = {}
config_params = {
    "internet_search" :{"default_val":"ddg_search"},
    "llm_provider"    :{"default_val":"groq"},
    "llm_model"       :{"default_val":"llama3-8b-8192"},
}

config = Config()
config._set_values_for_config(config_params)
llm_analyst = LLMAnalyst(query,config = config)

In [ ]:
research_state = llm_analyst.get_research_state()
research_state.report_md="Test"
print(research_state)
print(llm_analyst.cfg)

In [ ]:
llm_writer = LLMWriter.init(research_state)
print(llm_writer)

In [ ]:
empty_str = ""

if not empty_str:
    print("The string is empty.")
else:
    print("The string is not empty.")


In [ ]:
from llm_analyst.core.research_state import ResearchState
research_state = ResearchState(active_research_topic = "Test topic",
                                report_type = "detailed",
                                agent_type = "news agent",
                                agents_role_prompt="agent role")


research_state.dump("reseach_state.json")

In [ ]:
import sys
import os

sys.path.append('/Users/dan/Code/LLM/llm_analyst')
from llm_analyst.core.research_state import ResearchState
loaded = ResearchState.load("reseach_state.json")
print(loaded)

In [ ]:
import requests
import os
# Replace 'your_api_key_here' with your actual OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
headers = {
    'Authorization': f'Bearer {api_key}'
}

# API endpoint to get usage details
url = 'https://api.openai.com/v1/dashboard/billing/usage'

# Optional: Specify the date range for the usage data
params = {
    'start_date': '2024-05-01',
    'end_date': '2024-05-31'
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    usage_data = response.json()
    print("Usage data:", usage_data)
else:
    print(f"Failed to retrieve usage data. Status code: {response.status_code}")
    print("Response:", response.text)


In [ ]:
import hashlib
import os
import json

def compute_hash(directory_path):
    """Compute a single SHA-256 hash for the entire directory."""
    hasher = hashlib.sha256()
    
    for root, dirs, files in os.walk(directory_path):
        for file_name in sorted(files):  # Sort files to ensure consistent order
            file_path = os.path.join(root, file_name)
            with open(file_path, 'rb') as file:
                buffer = file.read()
                hasher.update(buffer)
                hasher.update(file_path.encode())  # Include file path in hash
    
    return hasher.hexdigest()

def has_changed(stored_hash, directory_path):
    """Check if the directory has changed by comparing hashes."""
    current_hash = compute_hash(directory_path)
    return current_hash != stored_hash


In [ ]:
directory_path = "/Users/dan/delme/articles"
dir_hash = compute_hash(directory_path)


In [ ]:
if has_changed(dir_hash, directory_path):
    print("has changed")
else:
    print("has not changed")

## Workon on Vector Store

In [1]:
import sys
import os
sys.path.append('/Users/dan/Code/LLM/llm_analyst')
from llm_analyst.core.config import Config

user_agent = ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
              "(KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 Edg/119.0.0.0")


os.environ['USER_AGENT'] = user_agent

In [2]:
# Read the URL CSV file and parse into cuncks
import json

def chunk_urls(url_file_nm):
    # Read the JSON file
    with open(url_file_nm, 'r') as file:
        urls = json.load(file)
    
    # Check if the data is a list
    if not isinstance(urls, list):
        raise ValueError("The JSON file does not contain a list of URLs.")
    
    # Chunk the list into sublists of 10 URLs each
    chunked_urls = [urls[i:i + 10] for i in range(0, len(urls), 10)]
    
    return chunked_urls

# Example usage
url_file_nm = 'data/urls.json'
chunked_urls = chunk_urls(url_file_nm)

# Print the chunked URLs
for idx, chunk in enumerate(chunked_urls):
    print(f"Chunk {idx + 1}: {chunk}")

Chunk 1: ['https://academic.oup.com/endo/article/151/9/4146/2456838', 'https://academic.oup.com/genetics/advance-article/doi/10.1093/genetics/iyab093/6298595', 'https://academic.oup.com/genetics/article/219/1/iyab093/6298595', 'https://academic.oup.com/genetics/article/219/1/iyab093/6298595', 'https://advances.sciencemag.org/content/6/26/eaaz9805', 'https://citeseerx.ist.psu.edu/document?repid=rep1&amp;type=pdf&amp;doi=77de5621618515ecc0ad8e9e5668e6f5477ce1bf', 'https://jcs.biologists.org/content/131/22/jcs217992', 'https://jcs.biologists.org/content/joces/130/24/4155.full.pdf', 'https://journals.biologists.com/jcs/article/114/24/4543/35724/Regulation-of-Golgi-structure-and-function-by-ARF', 'https://journals.biologists.com/jcs/article/123/8/1262/31619/Phosphatidylinositol-and-phosphatidylcholine']
Chunk 2: ['https://journals.sagepub.com/doi/10.1177/25152564211034424', 'https://link.springer.com/article/10.1007/s00125-021-05552-x', 'https://mcdb.yale.edu/people/shirin-bahmanyar', 'http

In [3]:
# # Process 1 URL and add to Vector Store

# from llm_analyst.documents.vector_store import VectorStore
# from llm_analyst.core.config import Config
# from llm_analyst.scrapers.scraper_methods import scrape_urls

# import requests
# from requests.packages.urllib3.exceptions import InsecureRequestWarning

# # Suppress only the specific warning
# requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


# config = Config()
# vector_store = VectorStore(config.cache_dir)
# try:
#     print(len(vector_store.vector_db))
# except:
#     print(0)
    
# vector_store.reset_vector_store_db()

    
# for urls in chunked_urls[0:1]:
#     print(f"Processing: {urls}")
#     documents = scrape_urls(urls)
#     await vector_store.add_documents(documents)

# try:
#     print(len(vector_store.vector_db))
# except:
#     print(0)
        
    


In [4]:
# docs = await vector_store.retrieve_docs_for_query("Extracellular Matrix Production")
# print(docs)

In [5]:
config = Config()
config.cache_dir

'/Users/dan/.cache/llm_analyst'

In [6]:
# # Load a vector_db locally in this notbeook 
# # Not using any framework code

# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
# import os

# persist_directory = "/Users/dan/.cache/llm_analyst/chroma_db"
# model_name = "sentence-transformers/all-MiniLM-l6-v2"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": False}
# embedding_model = HuggingFaceEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs,
# )

# if not os.path.exists(persist_directory):
#     os.makedirs(persist_directory)
# vector_db = Chroma(
#         persist_directory=persist_directory,
#         embedding_function=embedding_model,
#     )


In [7]:
biorxiv=["https://www.biorxiv.org/content/10.1101/2021.08.20.457118v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2021.08.27.457935v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2021.11.16.468906v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2022.01.26.477847v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2022.02.04.479101v2.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2022.02.14.480333v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2022.06.27.497668v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2022.09.15.508072v2.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2023.02.01.526230v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2023.02.13.528342v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2023.03.01.530598v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2023.03.01.530598v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2023.05.17.541061v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2023.08.31.555797v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2023.12.20.572417v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2024.03.07.583776v2.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2024.03.20.586037v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2024.05.01.592120v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2024.05.09.593427v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2024.05.20.594820v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2024.05.27.596058v1.full.pdf",
    "https://www.biorxiv.org/content/10.1101/2024.05.31.596797v1.full.pdf"]
 

In [8]:
pubmed_central=["https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10309186/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10525039/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10637267/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC10769218/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1237059/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2426940/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2488303/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3195608/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3836174/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4221251/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4635215/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5629314/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5746065/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5746065/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5789946/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5844805/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5994893/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6244289/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6540057/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6764485/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6764485/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6972958/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7072557/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7338047/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8212386/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8212386/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8431686/",
    "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9374436/"]


In [9]:
elife=["https://elifesciences.org/articles/24603",
    "https://elifesciences.org/articles/46595",
    "https://elifesciences.org/articles/55745",
    "https://elifesciences.org/articles/83884",
    "https://elifesciences.org/articles/92900"]

In [10]:
plos=["https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.3001051",
    "https://journals.plos.org/plosbiology/article?id=10.1371/journal.pbio.3001431",
    "https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1009602",
    "https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1009736",
    "https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1011078"]

In [11]:
nature=["https://www.nature.com/articles/ncb1774",
    "https://www.nature.com/articles/s41467-019-13867-y",
    "https://www.nature.com/articles/s41467-019-14038-9",
    "https://www.nature.com/articles/s41467-021-24081-0.epdf?sharing_token=FSX6rBpLLq2fr4Vx4vEbANRgN0jAjWel9jnR3ZoTv0PCmeZr94GGnK53maIYoeRVVWFWRDCKK3DD05aL90OBkD_uXqaE2zrSicRn2UrU7PX0qFqZPgntZzGG5NzOhXi7Ose54rYDxXEp5GT44HgE06odn22zTnl9pQW5upsHec8%3D|https://www.nature.com/articles/s41467-021-24081-0.epdf?sharing_token=FSX6rBpLLq2fr4[\u2026]gntZzGG5NzOhXi7Ose54rYDxXEp5GT44HgE06odn22zTnl9pQW5upsHec8%3D",
    "https://www.nature.com/articles/s41467-021-25398-6",
    "https://www.nature.com/articles/s41467-021-27279-4#disqus_thread",
    "https://www.nature.com/articles/s41467-022-28469-4",
    "https://www.nature.com/articles/s41467-022-30060-w",
    "https://www.nature.com/articles/s41467-022-32487-7",
    "https://www.nature.com/articles/s41467-024-45782-2",
    "https://www.nature.com/articles/s41556-022-00917-9",
    "https://www.nature.com/articles/s41556-023-01136-6#Bib1",
    "https://www.nature.com/articles/s41556-023-01180-2",
    "https://www.nature.com/articles/s41556-023-01287-6",
    "https://www.nature.com/articles/s41580-022-00524-4",
    "https://www.nature.com/articles/s41589-024-01575-8",
    "https://www.nature.com/articles/s41598-019-45020-6",
    "https://www.nature.com/articles/s41598-022-05340-6",
    "https://www.nature.com/articles/s41598-023-40705-5",
    "https://www.nature.com/articles/s42255-022-00706-6.epdf?sharing_token=XI30ZM2EGunwLedKAflYxtRgN0jAjWel9jnR3ZoTv0NIqjP5_r7Wh9CXPW5apSLqqo0IXTfF31sTM2rawNXbcLKtN3dQpUtN-IEh7NveS1DOHmmGKYG2ZkGkSuIuUCdwdRRp3KhJqywGB2MPG1MImBAGqRDUldbWsq_B9xVFSS0%3D|https://www.nature.com/articles/s42255-022-00706-6.epdf?sharing_token=XI30ZM2EGunwLe[\u2026]YG2ZkGkSuIuUCdwdRRp3KhJqywGB2MPG1MImBAGqRDUldbWsq_B9xVFSS0%3D",
    "https://www.nature.com/articles/s42255-023-00939-z?utm_source=dlvr.it&amp;utm_medium=twitter"]

In [12]:
 cell=["https://www.cell.com/biophysj/fulltext/S0006-3495(21)00991-7",
    "https://www.cell.com/biophysj/fulltext/S0006-3495(21)00991-7",
    "https://www.cell.com/cell-metabolism/fulltext/S1550-4131(21)00223-0?rss=yes&amp;utm_source=dlvr.it&amp;utm_medium=twitter",
    "https://www.cell.com/cell-metabolism/fulltext/S1550-4131(23)00130-4",
    "https://www.cell.com/cell-metabolism/fulltext/S1550-4131(23)00261-9#secsectitle0315",
    "https://www.cell.com/cell-reports/fulltext/S2211-1247(20)30821-4#.XvtjyPbShnI.twitter",
    "https://www.cell.com/cell-reports/fulltext/S2211-1247(20)31478-9",
    "https://www.cell.com/cell-reports/fulltext/S2211-1247(21)01770-8#.Yebhhfd3iDE.twitter",
    "https://www.cell.com/cell-reports/fulltext/S2211-1247(24)00686-7?rss=yes&amp;utm_source=dlvr.it&amp;utm_medium=twitter",
    "https://www.cell.com/developmental-cell/fulltext/S1534-5807%2824%2900180-1",
    "https://www.cell.com/developmental-cell/fulltext/S1534-5807(21)00601-8?rss=yes",
    "https://www.cell.com/developmental-cell/fulltext/S1534-5807(24)00180-1?rss=yes&amp;utm_source=dlvr.it&amp;utm_medium=twitter",
    "https://www.cell.com/neuron/fulltext/S0896-6273(22)00365-8#secsectitle0150",
    "https://www.cell.com/trends/biochemical-sciences/fulltext/S0968-0004(20)30067-0",
    "https://www.cell.com/trends/biochemical-sciences/fulltext/S0968-0004(20)30085-2",
    "https://www.cell.com/trends/cell-biology/fulltext/S0962-8924(20)30074-X",
    "https://www.cell.com/trends/cell-biology/fulltext/S0962-8924(20)30101-X"]


In [13]:
# A simple function to dump the scraped results

def dump_results(url, document):
    result = json.dumps(document, indent=4)
    out_file_nm = url.replace('https://','')
    out_file_nm = out_file_nm.replace('/','_')
    if len(out_file_nm) > 75:
        out_file_nm = out_file_nm[:75]
    print(f"Saving {url}")

    with open(f"data/out/{out_file_nm}.json", 'w', encoding='utf-8') as file:
        file.write(result)


In [14]:
# Usint the scrape_urls method get the data for the site

assigned_urls=cell[0:1]
from llm_analyst.scrapers.scraper_methods import scrape_urls
url_file_nm = 'data/urls.json'
with open(url_file_nm, 'r') as file:
    urls = json.load(file)
index = 1
for url in assigned_urls:
    print(f"{index} of {len(assigned_urls)} {url}")
    index +=1
    documents = [{"error":"Scrape error"}]
    try:
        documents = scrape_urls([url])
        if not documents:
            documents = [{"url":url}]
            url = "ERROR_"+url
    except:
        print("Error")
    
    dump_results(url, documents)


DEBUG:pyvirtualdisplay:version=3.0
DEBUG:pyvirtualdisplay.abstractdisplay:command: ['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '800x600x24', ':1051']
INFO:pyvirtualdisplay.abstractdisplay:Successfully started X with display ":1051".
DEBUG:pyvirtualdisplay.abstractdisplay:set $DISPLAY=:1051
DEBUG:selenium.webdriver.common.selenium_manager:Selenium Manager binary found at: /Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/selenium/webdriver/common/macos/selenium-manager
DEBUG:selenium.webdriver.common.selenium_manager:Executing process: /Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/selenium/webdriver/common/macos/selenium-manager --browser firefox --debug --language-binding python --output json
DEBUG:selenium.webdriver.common.selenium_manager:geckodriver not found in PATH
DEBUG:selenium.webdriver.common.selenium_manager:firefox not found in PATH
DEBUG:selenium.webdriver.common.selenium_manager:firefox not found in the system
DEBUG:sel

1 of 1 https://www.cell.com/biophysj/fulltext/S0006-3495(21)00991-7


DEBUG:selenium.webdriver.remote.remote_connection:Remote response: status=200 | data={"value":{"sessionId":"bdfcf2fd-c3c7-4e82-81fc-91c83c17affe","capabilities":{"acceptInsecureCerts":true,"browserName":"firefox","browserVersion":"127.0","moz:accessibilityChecks":false,"moz:buildID":"20240606181944","moz:debuggerAddress":"127.0.0.1:64685","moz:geckodriverVersion":"0.34.0","moz:headless":true,"moz:platformVersion":"23.5.0","moz:processID":97655,"moz:profile":"/var/folders/jq/7cv336vx66v_c3vj7vwrpt6c0000gn/T/rust_mozprofilefxNTOQ","moz:shutdownTimeout":60000,"moz:webdriverClick":true,"moz:windowless":false,"pageLoadStrategy":"normal","platformName":"mac","proxy":{},"setWindowRect":true,"strictFileInteractability":false,"timeouts":{"implicit":0,"pageLoad":300000,"script":30000},"unhandledPromptBehavior":"dismiss and notify","userAgent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:127.0) Gecko/20100101 Firefox/127.0"}}} | headers=HTTPHeaderDict({'content-type': 'application/json; char

No cookie consent button found or an error occurred: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16



DEBUG:selenium.webdriver.remote.remote_connection:Remote response: status=200 | data={"value":null} | headers=HTTPHeaderDict({'content-type': 'application/json; charset=utf-8', 'cache-control': 'no-cache', 'content-length': '14', 'date': 'Tue, 18 Jun 2024 19:27:52 GMT'})
DEBUG:selenium.webdriver.remote.remote_connection:Finished Request
DEBUG:pyvirtualdisplay.abstractdisplay:set $DISPLAY=/private/tmp/com.apple.launchd.YKxs2PyZJk/org.xquartz:0
DEBUG:pyvirtualdisplay.abstractdisplay:stdout=b''
DEBUG:pyvirtualdisplay.abstractdisplay:stderr=b''


Saving https://www.cell.com/biophysj/fulltext/S0006-3495(21)00991-7


In [ ]:
query = "Extracellular Matrix Production"
retriever = vector_db.as_retriever(
            search_type="similarity_score_threshold",
            search_kwargs={"k": 4, "score_threshold": 0.2},
        )
await retriever.ainvoke(query)

In [ ]:
import re

def clean(text):
  """Clean extra spaces in a string.

  Args:
      text: The string to clean.

  Returns:
      A string with extra spaces replaced by two spaces.
  """
  # Use regular expression to match 3 or more whitespace characters
  pattern = r"\s{3,}"
  return re.sub(pattern, "  ", text)

# Testing the function with a string with different spacings
text = """This is a string   with   
extra  spaces
."""
cleaned_text = clean(text)
print(cleaned_text)



In [ ]:
from pyvirtualdisplay import Display  
from selenium import webdriver  
display = Display(visible=0, size=(800, 600))  
display.start()  
browser = webdriver.Firefox()  
browser.get('https://www.cell.com/biophysj/fulltext/S0006-3495(21)00991-7')  

page_source = browser.page_source
print(page_source)
#browser.quit()  
#display.stop()


In [ ]:
from pyvirtualdisplay import Display
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re

# Start virtual display
display = Display(visible=0, size=(800, 600))
display.start()

# Set up the browser
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
browser = webdriver.Firefox(options=options)

# Load the web page
browser.get('https://www.cell.com/biophysj/fulltext/S0006-3495(21)00991-7')

try:
    # Wait for the cookie consent button to be present and clickable
    cookie_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Accept')]"))
    )
    # Click the cookie consent button
    cookie_button.click()
except Exception as e:
    print("No cookie consent button found or an error occurred:", e)

# Get the whole web page content
page_source = browser.page_source
soup = BeautifulSoup(page_source, 'html.parser')

# Extract text content and print it
text_content = soup.get_text()
pattern = r"\s{3,}"
text_content = re.sub(pattern, "  ", text_content)
        
print(text_content)

# Clean up
browser.quit()
display.stop()
